## Nvidia GPU

In [1]:
!nvidia-smi

Mon Dec  9 02:04:14 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8              8W /  115W |      52MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Frameworks

In [2]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Nov_30_19:15:10_Pacific_Standard_Time_2020
Cuda compilation tools, release 11.2, V11.2.67
Build cuda_11.2.r11.2/compiler.29373293_0
torch:  1.4 ; cuda:  cu92


## Dataset

In [ ]:
path: ../

train: train
val: valid
test: test

nc: 1
names: ['face']

# roboflow:
#  workspace: uniform-fhonp
#  project: face-detection-quvrj
#  version: 1
#  license: CC BY 4.0
#  url: https://universe.roboflow.com/uniform-fhonp/face-detection-quvrj/dataset/1

## Training

In [ ]:
from ultralytics import YOLO

if __name__ == "__main__":
    model = YOLO("yolov8n.pt")
    model.train(data="../FaceData/data.yaml", epochs=100)
    model.val()


## Evaluation

In [ ]:
from ultralytics import YOLO

if __name__ == "__main__":
    model = YOLO("./face_detector.pt")
    model.val()


## Image Prediction 

In [ ]:
# coding:utf-8
from ultralytics import YOLO
import cv2
import os


def validate_file_path(file_path, base_dir="."):
    """
    Validate if a file path is within a specific base directory.
    Prevents directory traversal attacks.
    """
    abs_path = os.path.abspath(file_path)
    base_dir = os.path.abspath(base_dir)

    if not abs_path.startswith(base_dir):
        raise ValueError(f"Invalid file path: {file_path}")

    return abs_path


def img_cvread(img_path):
    """Read the image and return the image array"""
    return cv2.imread(img_path)


def face_detect(cv_img, face_model):
    """Perform face detection and return detected face images and their locations"""
    results = face_model(cv_img)
    faces = []
    locations = []
    for result in results:
        for bbox in result.boxes:
            x1, y1, x2, y2 = map(int, bbox.xyxy[0].cpu().numpy())
            face = cv_img[y1:y2, x1:x2]
            faces.append(face)
            locations.append((x1, y1, x2, y2))
    return cv_img, faces, locations


def adjust_parameters(width, height):
    """Adjust parameters based on image size"""
    base_width = 640
    scale = min(width / base_width, height / base_width)
    box_thickness = int(10 * scale)
    return box_thickness


if __name__ == '__main__':
    base_dir = "."  # Define a base directory to validate file paths

    img_path = ""       # Input image file path
    output_path = ""    # Output image file path

    # Validate file paths
    try:
        img_path = validate_file_path(img_path, base_dir)
        output_path = validate_file_path(output_path, base_dir)
    except ValueError as e:
        print(e)
        exit()

    # Path to the face detection model
    face_model_path = 'face_detector.pt'

    # Load the face detection model
    face_model = YOLO(face_model_path, task='detect')

    cv_img = img_cvread(img_path)
    if cv_img is None:
        print(f"Error: Could not read image from path: {img_path}")
        exit()

    height, width = cv_img.shape[:2]

    # Adjust parameters
    box_thickness = adjust_parameters(width, height)

    # Perform face detection
    face_cvimg, faces, locations = face_detect(cv_img, face_model)

    if faces:
        for i in range(len(faces)):
            left, top, right, bottom = locations[i]
            # Draw rectangle around the detected face
            face_cvimg = cv2.rectangle(face_cvimg, (left, top), (right, bottom), (50, 50, 250), box_thickness)

    # Save the predicted image
    cv2.imwrite(output_path, face_cvimg)

    # Display the predicted image
    cv2.imshow('yolov8_detections', face_cvimg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


## Video Prediction

In [ ]:
# coding:utf-8
from ultralytics import YOLO
import cv2
import os
from moviepy.editor import VideoFileClip, AudioFileClip


def validate_file_path(file_path, base_dir="."):
    """
    Validate if a file path is within a specific base directory.
    Prevents directory traversal attacks.
    """
    abs_path = os.path.abspath(file_path)
    base_dir = os.path.abspath(base_dir)

    if not abs_path.startswith(base_dir):
        raise ValueError(f"Invalid file path: {file_path}")

    return abs_path


def face_detect(cv_img, face_model):
    """Perform face detection and return detected face images and their locations"""
    results = face_model(cv_img)
    faces = []
    locations = []
    for result in results:
        for bbox in result.boxes:
            x1, y1, x2, y2 = map(int, bbox.xyxy[0].cpu().numpy())
            face = cv_img[y1:y2, x1:x2]
            faces.append(face)
            locations.append((x1, y1, x2, y2))
    return cv_img, locations


def process_frame(frame, face_model):
    """Process video frame for face detection"""
    face_cvimg, locations = face_detect(frame, face_model)

    if locations:
        for (left, top, right, bottom) in locations:
            # Draw rectangle around detected faces
            face_cvimg = cv2.rectangle(face_cvimg, (left, top), (right, bottom), (50, 50, 250), 2)

    return face_cvimg


if __name__ == '__main__':
    base_dir = "."  # Define a base directory to validate file paths

    video_path = ""     # Input video file path
    output_path = ""    # Output video file path

    # Validate file paths
    try:
        video_path = validate_file_path(video_path, base_dir)
        output_path = validate_file_path(output_path, base_dir)
    except ValueError as e:
        print(e)
        exit()

    # Path to the face detection model
    face_model_path = 'face_detector.pt'

    # Load the face detection model
    face_model = YOLO(face_model_path)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file: {video_path}")
        exit()

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Open video writer
    out = cv2.VideoWriter('temp_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process each frame
        processed_frame = process_frame(frame, face_model)

        # Display the result
        cv2.imshow('Face Detection', processed_frame)

        # Write the processed frame
        out.write(processed_frame)

        # Exit on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Combine processed video with original audio
    video_clip = VideoFileClip("temp_video.mp4")
    audio_clip = AudioFileClip(video_path)
    final_clip = video_clip.set_audio(audio_clip)
    final_clip.write_videofile(output_path, codec='libx264')


##  Real-time Prediction from Camera

In [ ]:
# coding:utf-8
from ultralytics import YOLO
import cv2


def face_detect(cv_img, face_model):
    """Perform face detection and return detected face images and their locations"""
    results = face_model(cv_img)
    faces = []
    locations = []
    for result in results:
        for bbox in result.boxes:
            x1, y1, x2, y2 = map(int, bbox.xyxy[0].cpu().numpy())
            face = cv_img[y1:y2, x1:x2]
            faces.append(face)
            locations.append((x1, y1, x2, y2))
    return cv_img, locations


def adjust_parameters(width, height):
    """Adjust parameters based on image size"""
    base_width = 640
    scale = min(width / base_width, height / base_width)
    box_thickness = int(10 * scale)
    return box_thickness


if __name__ == '__main__':
    # Path to the face detection model
    face_model_path = 'face_detector.pt'

    # Load the face detection model
    face_model = YOLO(face_model_path, task='detect')

    # Open the camera, 0 indicates the default camera
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open camera.")
        exit()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to read frame.")
            break

        height, width = frame.shape[:2]

        # Adjust parameters
        box_thickness = adjust_parameters(width, height)

        # Perform face detection
        face_cvimg, locations = face_detect(frame, face_model)

        if locations:
            for (left, top, right, bottom) in locations:
                # Draw rectangle around detected faces
                face_cvimg = cv2.rectangle(face_cvimg, (left, top), (right, bottom), (50, 50, 250), box_thickness)

        # Display the frame with detection results
        cv2.imshow('Face Detection System', face_cvimg)

        # Exit the loop when 'q' key is pressed
        if cv2.waitKey(1) == ord('q'):
            break

    # Release the camera resource
    cap.release()
    # Close all OpenCV windows
    cv2.destroyAllWindows()
